# Gain estimation using photo-statistics method    

calin/examples/calib/gain estimation from photostatistics.ipynb - Stephen Fegan - 2017-03-27

Copyright 2017, Stephen Fegan <sfegan@llr.in2p3.fr>
LLR, Ecole polytechnique, CNRS/IN2P3, Universite Paris-Saclay

This file is part of "__calin__". "__calin__" is free software: you can redistribute it and/or modify it under the
terms of the GNU General Public License version 2 or later, as published by
the Free Software Foundation. "__calin__" is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.

## Introduction

This notebook demonstrates how to use the diagnostics results written by __compute\_diagnostics.py__ to calculate the gain of the channels using the photo-statistics method. The calculation uses uses equation 30 of MST-CAM-TN-0060, which is appropriate when the intrinsic variance of the flasher is small, i.e. when Poisson fluctuations in the channels dominate. The intrinsic flasher variance is compensated for as described in the memo, however if the flasher resolution exceeds 3% other methods may give better results.

## Get diagnostics results from SQL database

Open the SQL diagnostics results previously written by __compute\_diagnostics.py__ and load the results.

In [ ]:
%pylab inline
import calin.ix.scripts.compute_diagnostics
import calin.io.sql_transceiver
import calin.diagnostics.waveform
import calin.diagnostics.functional
import calin.plotting

In [ ]:
sql = calin.io.sql_transceiver.SQLite3Transceiver("/CTA/diagnostics.sqlite",
  calin.io.sql_transceiver.SQLite3Transceiver.READ_ONLY)
diagnostics = calin.ix.scripts.compute_diagnostics.Results()
sql.retrieve_by_oid("diagnostics_results", 1, diagnostics)
del sql

In [ ]:
cfg = diagnostics.run_config()
clo = diagnostics.command_line_options()

## Illustrate signal & background windows

Draw the average trace over all channels and illustrate the signal and background windows. This is done as a sanity check.

In [ ]:
wfs  = diagnostics.waveform_stats()
wf_mean = zeros(cfg.num_samples())
wf_var = zeros(cfg.num_samples())
for ich in  range(0,wfs.high_gain_size()):
    wf_mean += calin.diagnostics.waveform.WaveformStatsVisitor.waveform_mean(wfs.high_gain(ich))
    wf_var += calin.diagnostics.waveform.WaveformStatsVisitor.waveform_var(wfs.high_gain(ich))
wf_mean /= wfs.high_gain_size()
wf_var /= wfs.high_gain_size()**2

In [ ]:
errorbar(frange(len(wf_mean),closed=False),wf_mean,sqrt(wf_var),fmt='k.-')
a=axis()
gca().add_patch(Rectangle((clo.sig_window_start(), a[2]), clo.window_size(), a[3]-a[2], facecolor='#ffeeee'))
axvline(clo.sig_window_start(),color='r')
axvline(clo.sig_window_start()+clo.window_size(),color='r')
text(clo.sig_window_start()+clo.window_size()/2,a[2]*0.975+a[3]*0.025,'Signal',ha='center',va='bottom',color='r')
gca().add_patch(Rectangle((clo.bkg_window_start(), a[2]), clo.window_size(), a[3]-a[2], facecolor='#eeeeff'))
axvline(clo.bkg_window_start(),color='b')
axvline(clo.bkg_window_start()+clo.window_size(),color='b')
text(clo.bkg_window_start()+clo.window_size()/2,a[3]*0.975+a[2]*0.025,'Background',ha='center',va='top',color='b')
xlabel('Sample number [ns]')
ylabel('Average pulse amplitude [DC]')

## Calculate the gain of the high-gain channel

Extract the mean and variance of the signal and background regions and decompose them to calculate the common-mode component that can be attributed to the intrinsic variance of the flasher, and the component in each channel that is independent of this.

Calculate the gain in each channel from this accounting for the excess-noise fraction of the PMT single-electron multiplier, which must be specified.

In [ ]:
enf = 1.14

In [ ]:
smi=calin.diagnostics.functional.channel_mean(diagnostics.sig_stats().high_gain())
bmi=calin.diagnostics.functional.channel_mean(diagnostics.bkg_stats().high_gain())
svi=calin.diagnostics.functional.channel_var(diagnostics.sig_stats().high_gain())
bvi=calin.diagnostics.functional.channel_var(diagnostics.bkg_stats().high_gain())

In [ ]:
svi_indep,sv_cm=calin.diagnostics.functional.decompose_channel_independent_and_common_var(\
        diagnostics.sig_stats().high_gain())

In [ ]:
g = (svi - bvi - sv_cm*((smi-bmi)/mean(smi-bmi))**2)/(smi-bmi)/enf

## Print and display the results

In [ ]:
for i,l in enumerate([g[i:i + 7] for i in range(0, len(g), 7)]):
    print("| Module %-2d"%cfg.configured_module_id(i),'|',' | '.\
          join(map(lambda x: '%5.3f'%x, l)),'|')

In [ ]:
calin.plotting.plot_camera(g, cfg.camera_layout(), cfg.configured_channel_id_view())
title('Gain in high-gain channels')

## Calculate the gain of the low-gain channel

This is mostly only for fun, or more accurately as a sanity check. A better way is probably to estimate the high-to-low gain ratio using the position of the mean signal in each channel and extrapolate from the absolute gain high-gain channels. This is effectively done at the very end.

In [ ]:
lg_smi=calin.diagnostics.functional.channel_mean(diagnostics.sig_stats().low_gain())
lg_bmi=calin.diagnostics.functional.channel_mean(diagnostics.bkg_stats().low_gain())
lg_svi=calin.diagnostics.functional.channel_var(diagnostics.sig_stats().low_gain())
lg_bvi=calin.diagnostics.functional.channel_var(diagnostics.bkg_stats().low_gain())

In [ ]:
lg_svi_indep,lg_sv_cm=calin.diagnostics.functional.decompose_channel_independent_and_common_var(\
        diagnostics.sig_stats().low_gain())

In [ ]:
lg_g = (lg_svi - lg_bvi - lg_sv_cm*((lg_smi-lg_bmi)/mean(lg_smi-lg_bmi))**2)/(lg_smi-lg_bmi)/enf

In [ ]:
for i,l in enumerate([lg_g[i:i + 7] for i in range(0, len(lg_g), 7)]):
    print("| Module %-2d"%cfg.configured_module_id(i),'|',' | '.\
          join(map(lambda x: '%5.3f'%x, l)),'|')

In [ ]:
calin.plotting.plot_camera(lg_g, cfg.camera_layout(), cfg.configured_channel_id_view())
title('Gain in low-gain channels')

## Calculate the high-to-low gain ratio

- First, by dividing the absolute gains calculated above
- Secondly, by dividing the means of the signal histograms

In [ ]:
g_ratio = g/lg_g
for i,l in enumerate([g_ratio[i:i + 7] for i in range(0, len(g_ratio), 7)]):
    print("| Module %-2d"%cfg.configured_module_id(i),'|',' | '.\
          join(map(lambda x: '%5.3f'%x, l)),'|')

In [ ]:
rg_ratio = (smi-bmi)/(lg_smi-lg_bmi)
for i,l in enumerate([rg_ratio[i:i + 7] for i in range(0, len(rg_ratio), 7)]):
    print("| Module %-2d"%cfg.configured_module_id(i),'|',' | '.\
          join(map(lambda x: '%5.3f'%x, l)),'|')

In [ ]:
plot(g_ratio, rg_ratio,'x')
axis('square')
xlabel('Absolute gain raio')
ylabel('Relative gain raio')